Import statements

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model, preprocessing
import os
import glob
import pandas as pd
import nltk
from nltk import tokenize
from nltk.stem import WordNetLemmatizer
import re 
# nltk.download('stopwords')
# nltk.download()
from nltk.corpus import stopwords 

In [0]:
nltk.download('punkt')
nltk.download('all')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /r

True

Compile and run from here.

To load the train and validation data from TSV files.

Arguments: train file, validation file

Returns:

tr_st: Training sentences
tr_ft: Training Future Labels
tr_pt: Trainiing past labels
val_st: Validation sentences
val_ft: Validation Future Labels
val_pt: Validation past labels

In [0]:
def load_train_val(train,val):
  df= pd.read_csv(train,delimiter='\t',encoding='utf-8')
  tr_st=np.array(df['sentence'].values)
  tr_ft=np.array(df['Future'].values)
  tr_pt=np.array(df['Past'].values)
  df= pd.read_csv(val,delimiter='\t',encoding='utf-8')
  val_st=np.array(df['sentence'].values)
  val_ft=np.array(df['Future'].values)
  val_pt=np.array(df['Past'].values)
  return tr_st,tr_ft,tr_pt, val_st,val_ft,val_pt


Method to load the test data from TSV file.

Arguments: test file

Returns:

ts_st: Test sentences
ts_ft: Test Future Labels
ts_pt: Test past labels

In [0]:
def load_test(test):
  df= pd.read_csv(test,delimiter='\t',encoding='utf-8')
  ts_st=np.array(df['sentence'].values)
  ts_ft=np.array(df['Future'].values)
  ts_pt=np.array(df['Past'].values)
  return ts_st,ts_ft,ts_pt

To preprocess sentences:
Removing some special characters, spaces and tabs, and conversion to lower case

In [0]:
def preprocess_sentences(X):
  documents = []
  stemmer = WordNetLemmatizer()
  for sen in range(0, len(X)):
    document = re.sub(r'\W', ' ', str(X[sen]))
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'^"', '', document)
    document = re.sub(r'"$', '', document)
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub(r'^b\s+', '', document)
    document = document.lower()
    #     document = document.split()
    #     document = [stemmer.lemmatize(word) for word in document]
    #     document = ' '.join(document)
    documents.append(document)
  print(len(documents))
  return documents

Converts sentences to lists of features

Uncomment code as needed for features



In [0]:
def sent_to_features(documents):
  feature_docs=[]
  for document in documents:
    tokens=nltk.word_tokenize(document)
    bigrams = list(nltk.bigrams(tokens))
    #pos_bigrams = list(nltk.bigrams([pos for (word,pos) in nltk.pos_tag(tokens)]))
    document=[word for (word,pos) in nltk.pos_tag(tokens)]#+' '+pos
#     document.extend([w1+'_'+w2 for (w1,w2) in bigrams])
    #     document.extend([w1+'_'+w2 for (w1,w2) in pos_bigrams])
    #     document.extend([pos for (word,pos) in nltk.pos_tag(tokens)])
    #     document = document.split()
    #     document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    feature_docs.append(document)
  return feature_docs

This is not necesasry unless the code is being run on a Colab notebook, and the data is stored on Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Path to the location where the data resides

In [0]:
datapath = 'gdrive/My Drive/data/'

Loading the train and validation data

Preprocessing the data

Feature extraction if necessary

In [0]:
#loading train and validation data
trainfile=datapath+'train.txt'
valfile=datapath+'validation.txt'
tr_st,tr_ft,tr_pt, val_st,val_ft,val_pt=load_train_val(trainfile,valfile)

#preprocessing
tr_st = preprocess_sentences(tr_st)
val_st = preprocess_sentences(val_st)

#feature extraction
tr_st = sent_to_features(tr_st)
val_st = sent_to_features(val_st)

print('Train size: ',len(tr_st))
print('Val size: ',len(val_st))

1776
592
Train size:  1776
Val size:  592


In [0]:
print(tr_st[1])

NameError: ignored

Imports necessary for LSTM 

In [0]:
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, CuDNNLSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras import optimizers

# from tensorflow.keras.backend import set_session
import keras.backend as K
import nltk
import string
import numpy as np
import pandas as pd
import re
from tqdm import tqdm


Using TensorFlow backend.


Creating a Keras Tokenizer and fitting it on train and validation data

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(np.concatenate((tr_st,val_st)))

In [0]:
from gensim.models import Word2Vec

pos_model = Word2Vec.load(datapath+'model_300d.bin')
# print(pos_model)
words = list(pos_model.wv.vocab)
# print(words)
# print(len(words))
# print(pos_model['NN'])



Word2Vec(vocab=33, size=300, alpha=0.025)
['IN', 'PRP', 'RB', 'VB', 'DT', 'JJ', 'NNS', 'CD', 'NN', 'MD', 'PRP$', 'CC', 'VBD', 'VBG', 'VBN', 'VBZ', 'WDT', 'JJS', 'TO', 'JJR', 'VBP', 'RP', 'RBR', 'WP$', 'WP', 'WRB', 'EX', 'PDT', 'RBS', '$', 'FW', 'NNP', 'NNPS']
33


Initializing vocabulary size

In [0]:
vocabulary_size = len(tokenizer.word_index)+1
# print(tokenizer.word_counts)
# print(tokenizer.document_count)
# print(tokenizer.word_index)
# print(tokenizer.word_docs)


In [0]:
print(vocabulary_size)

6744


In [0]:
# #feature extraction
# tr_st = sent_to_features(tr_st)
# val_st = sent_to_features(val_st)

Extracting sequences from train and validation data, and padding them to a maximum length

In [0]:

# vocabulary_size = 7000
# tokenizer.fit_on_texts(np.concatenate((tr_st,val_st)))

sequences = tokenizer.texts_to_sequences(tr_st)
tr_data = pad_sequences(sequences, maxlen=145)
sequences = tokenizer.texts_to_sequences(val_st)
val_data = pad_sequences(sequences, maxlen=145)

print(len(tr_data))
print(len(val_data))

1776
592


Loading word embeddings

In [0]:
f = open(datapath+'vecs.txt',encoding='utf-8', errors='ignore')
embedding_values = {}
embedding_size=400
i=0;
for line in tqdm(f):
  if(i==0):
    i=i+1
    continue
#   line= re.sub(r'\s', '\t', line)
  line= re.sub(r'\s+?$', '', line)
  value = line.split(' ')
  word = value[0]
#   if (word=='<unk>'):
#     print(value[1:])
#     break
#   print(len(value[1:]))
#   print(value[1:])
  val=[float(em) for em in value[1:]]
  coef = np.array(val,dtype = 'float32')
  embedding_values[word]=coef




0it [00:00, ?it/s]


422it [00:00, 4215.99it/s]


838it [00:00, 4198.06it/s]


1257it [00:00, 4195.13it/s]


1672it [00:00, 4180.25it/s]


2088it [00:00, 4172.65it/s]


2506it [00:00, 4173.58it/s]


2923it [00:00, 4170.24it/s]


3306it [00:00, 4033.27it/s]


3705it [00:00, 4018.25it/s]


4100it [00:01, 3995.72it/s]


4508it [00:01, 4018.00it/s]


4931it [00:01, 4077.68it/s]


5344it [00:01, 4091.68it/s]


5763it [00:01, 4118.86it/s]


6173it [00:01, 4112.41it/s]


6588it [00:01, 4122.65it/s]


7005it [00:01, 4136.63it/s]


7418it [00:01, 4084.06it/s]


7826it [00:01, 4033.14it/s]


8246it [00:02, 4079.48it/s]


8664it [00:02, 4108.93it/s]


9086it [00:02, 4140.59it/s]


9505it [00:02, 4153.98it/s]


9932it [00:02, 4186.26it/s]


10351it [00:02, 4182.47it/s]


10773it [00:02, 4191.32it/s]


11193it [00:02, 4191.70it/s]


11617it [00:02, 4204.31it/s]


12038it [00:02, 4077.56it/s]


12449it [00:03, 4084.94it/s]


12873it [00:03, 4127.66it/s]


13290it [00:03, 4139.90it/s]


13710it [0

Adding embedding values of POS tags

In [0]:
for word in words:
  coef=np.array(pos_model[word],dtype = 'float32')
  embedding_values[word]=coef

In [0]:
len(embedding_values)

418128

Creating an embedding matrix for words 

In [0]:
embedding_matrix = np.zeros((vocabulary_size,embedding_size))
for word,i in tqdm(tokenizer.word_index.items()):
  values = embedding_values.get(word)
  if values is not None:
    embedding_matrix[i] = values




  0%|          | 0/6743 [00:00<?, ?it/s]


100%|██████████| 6743/6743 [00:00<00:00, 288096.97it/s]

In [0]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

BiLSTM for Future task

In [0]:
#Future
K.clear_session()
model = Sequential()
model.add(Embedding(vocabulary_size,embedding_size, input_length = 145,weights = [embedding_matrix],trainable = False))
# model.add(Embedding(vocabulary_size, 145, input_length=145))
model.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2)))

model.add(Dense(64,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

adg= optimizers.Adagrad(lr=0.01)
model.compile(loss='binary_crossentropy', optimizer=adg, metrics=['accuracy'])

model.summary()
model.fit(tr_data, tr_ft, validation_data=(val_data,val_ft), epochs=20,batch_size=64) 
# model.fit(np.concatenate((tr_data,val_data)), np.concatenate((tr_ft,val_ft)), epochs=10,batch_size=64)#  validation_data=(val_data,val_ft),

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 145, 400)          2697600   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 290)               633360    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                18624     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 3,349,649
Trainable params: 652,049
Non-trainable params: 2,697,600
_________________________________________________________________
Train on 1776 samples, validate on 592 samples
Epoch 1/20
1776/1776 [==============================] - 16s 9ms/step - loss: 0.6959 - acc: 0.5822 - val_loss: 0.6963 - val_acc: 0.5726
Epoch 2/20
1776/1776 [==============================] - 16s 9ms/step - loss:

Stacked BiLSTM for Future task

In [0]:
#Future LSTM2
K.clear_session()
model2 = Sequential()
model2.add(Embedding(vocabulary_size,embedding_size, input_length = 145,weights = [embedding_matrix],trainable = False))
# model.add(Embedding(vocabulary_size, 145, input_length=145))
model2.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
model2.add(Bidirectional(LSTM(145, dropout=0.1, recurrent_dropout=0.1)))
model2.add(Dense(64,activation = 'relu'))
model2.add(Dense(1, activation='sigmoid'))
adg= optimizers.Adagrad(lr=0.01)
model2.compile(loss='binary_crossentropy', optimizer=adg, metrics=['accuracy'])
# model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 145, 400)          2697600   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 145, 290)          633360    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 290)               505760    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                18624     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 3,855,409
Trainable params: 1,157,809
Non-trainable params: 2,697,600
_________________________________________________________________


In [0]:
model2.fit(tr_data, tr_ft, validation_data=(val_data,val_ft), epochs=20,batch_size=64)  #epoch 15 batch 128 no relu hidden   epoch 7 batch 128 w/ relu hidden

Train on 1776 samples, validate on 592 samples
Epoch 1/20
1776/1776 [==============================] - 33s 18ms/step - loss: 0.7152 - acc: 0.5845 - val_loss: 0.6527 - val_acc: 0.6909
Epoch 2/20
1776/1776 [==============================] - 31s 18ms/step - loss: 0.6187 - acc: 0.6751 - val_loss: 0.5331 - val_acc: 0.7635
Epoch 3/20
1776/1776 [==============================] - 31s 18ms/step - loss: 0.5336 - acc: 0.7601 - val_loss: 0.4757 - val_acc: 0.7905
Epoch 4/20
1776/1776 [==============================] - 49s 28ms/step - loss: 0.5056 - acc: 0.7691 - val_loss: 0.4632 - val_acc: 0.7956
Epoch 5/20
1776/1776 [==============================] - 57s 32ms/step - loss: 0.4708 - acc: 0.7922 - val_loss: 0.4832 - val_acc: 0.7889
Epoch 6/20
1776/1776 [==============================] - 56s 32ms/step - loss: 0.4599 - acc: 0.7973 - val_loss: 0.5212 - val_acc: 0.7584
Epoch 7/20
1776/1776 [==============================] - 58s 33ms/step - loss: 0.4475 - acc: 0.8029 - val_loss: 0.4811 - val_acc: 0.7838
E

Training on whole training data after parameter tuning, in order to use the model to evaluate the test data

In [0]:
model.fit(np.concatenate((tr_data,val_data)), np.concatenate((tr_ft,val_ft)), epochs=8,batch_size=64)

Loading the Test data

preprocessing the data

Extracting sequences

Padding the sequences



In [0]:
testfile=datapath+'test.txt'
ts_st,ts_ft,ts_pt=load_test(testfile)

#preprocessing
ts_st = preprocess_sentences(ts_st)

#feature extraction
#ts_st = sent_to_features(ts_st)
sequences = tokenizer.texts_to_sequences(ts_st)
ts_data = pad_sequences(sequences, maxlen=145)
print(ts_data.shape)
print(ts_ft.shape)
# test_loss, test_acc = model.evaluate(np.append((ts_data,ts_ft),1))
# print('Test Loss: {}'.format(test_loss))
# print('Test Accuracy: {}'.format(test_acc))


# print('Accuaracy = {0:f}'.format(acc))

593
(593, 145)
(593,)


Evaluating the test data for future task 

In [0]:

scores = model2.evaluate(ts_data, ts_ft, verbose=0)

print("Accuracy: %.2f%%" % (scores[1]*100))


Accuracy: 80.78%


In [0]:
print(ts_data.shape)

(593, 145)


Generating a classification report for the test data for future task

In [0]:
from sklearn.metrics import classification_report
y_pred = model2.predict(ts_data, batch_size=64, verbose=1)
y_pred =(y_pred>0.5)

print(classification_report(ts_ft, y_pred))

593/593 [==============================] - 5s 8ms/step
              precision    recall  f1-score   support

         0.0       0.84      0.75      0.79       293
         1.0       0.78      0.86      0.82       300

   micro avg       0.81      0.81      0.81       593
   macro avg       0.81      0.81      0.81       593
weighted avg       0.81      0.81      0.81       593



In [0]:
# scores = model2.evaluate(ts_data, ts_ft, verbose=0)

# print("Accuracy: %.2f%%" % (scores[1]*100))
# y_pred = model2.predict(ts_data, batch_size=64, verbose=1)
# y_pred =(y_pred>0.5)

# print(classification_report(ts_ft, y_pred))


BiLSTM model for Past task

In [0]:
#Past

K.clear_session()
model = Sequential()
model.add(Embedding(vocabulary_size,embedding_size, input_length = 145,weights = [embedding_matrix],trainable = False))
# model.add(Embedding(vocabulary_size, 145, input_length=145))
model.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 145, 400)          2697600   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 290)               633360    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                18624     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 3,349,649
Trainable params: 652,049
Non-trainable params: 2,697,600
_________________________________________________________________


In [0]:
model.fit(tr_data, tr_pt, validation_data=(val_data,val_pt), epochs=20,batch_size=64) #eposh 8 batch 128 no relu hidden

Train on 1776 samples, validate on 592 samples
Epoch 1/20
1776/1776 [==============================] - 16s 9ms/step - loss: 0.6599 - acc: 0.6194 - val_loss: 0.6766 - val_acc: 0.5946
Epoch 2/20
1776/1776 [==============================] - 15s 8ms/step - loss: 0.6077 - acc: 0.6796 - val_loss: 0.6205 - val_acc: 0.6841
Epoch 3/20
1776/1776 [==============================] - 16s 9ms/step - loss: 0.5470 - acc: 0.7382 - val_loss: 0.5944 - val_acc: 0.7331
Epoch 4/20
1776/1776 [==============================] - 15s 9ms/step - loss: 0.5212 - acc: 0.7646 - val_loss: 0.4965 - val_acc: 0.7889
Epoch 5/20
1776/1776 [==============================] - 15s 8ms/step - loss: 0.4529 - acc: 0.8102 - val_loss: 0.5104 - val_acc: 0.7855
Epoch 6/20
1776/1776 [==============================] - 15s 8ms/step - loss: 0.4591 - acc: 0.8035 - val_loss: 0.5217 - val_acc: 0.7652
Epoch 7/20
1776/1776 [==============================] - 16s 9ms/step - loss: 0.4431 - acc: 0.8176 - val_loss: 0.4822 - val_acc: 0.7838
Epoch 8/

Stacked BiLSTM for Past task

In [0]:
#Past LSTM2
K.clear_session()
model2 = Sequential()
model2.add(Embedding(vocabulary_size,embedding_size, input_length = 145,weights = [embedding_matrix],trainable = False))
# model.add(Embedding(vocabulary_size, 145, input_length=145))
model2.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
model2.add(Bidirectional(LSTM(145, dropout=0.1, recurrent_dropout=0.1)))
model2.add(Dense(64,activation = 'relu'))
model2.add(Dense(1, activation='sigmoid'))
adg= optimizers.Adagrad(lr=0.01)
model2.compile(loss='binary_crossentropy', optimizer=adg, metrics=['accuracy'])
# model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 145, 400)          2697600   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 145, 290)          633360    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 290)               505760    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                18624     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 3,855,409
Trainable params: 1,157,809
Non-trainable params: 2,697,600
_________________________________________________________________


In [0]:
model2.fit(tr_data, tr_pt, validation_data=(val_data,val_pt), epochs=20,batch_size=64) #eposh 8 batch 128 no relu hidden

Train on 1776 samples, validate on 592 samples
Epoch 1/20
1776/1776 [==============================] - 33s 18ms/step - loss: 0.6819 - acc: 0.6053 - val_loss: 0.6515 - val_acc: 0.5946
Epoch 2/20
1776/1776 [==============================] - 31s 18ms/step - loss: 0.6707 - acc: 0.6475 - val_loss: 0.6457 - val_acc: 0.6030
Epoch 3/20
1776/1776 [==============================] - 30s 17ms/step - loss: 0.5815 - acc: 0.7179 - val_loss: 0.5316 - val_acc: 0.7432
Epoch 4/20
1776/1776 [==============================] - 31s 17ms/step - loss: 0.5231 - acc: 0.7573 - val_loss: 0.5295 - val_acc: 0.7770
Epoch 5/20
1776/1776 [==============================] - 31s 18ms/step - loss: 0.4830 - acc: 0.7810 - val_loss: 0.5458 - val_acc: 0.7568
Epoch 6/20
1776/1776 [==============================] - 31s 18ms/step - loss: 0.4957 - acc: 0.7917 - val_loss: 0.5043 - val_acc: 0.7838
Epoch 7/20
1776/1776 [==============================] - 31s 18ms/step - loss: 0.4532 - acc: 0.8108 - val_loss: 0.4848 - val_acc: 0.7838
E

Evaluating the test data for future task 

In [0]:

scores = model.evaluate(ts_data, ts_pt, verbose=0)

print("Accuracy: %.2f%%" % (scores[1]*100))


Generating a classification report for the test data for future task

In [0]:
from sklearn.metrics import classification_report
y_pred = model.predict(ts_data, batch_size=64, verbose=1)
y_pred =(y_pred>0.5)

print(classification_report(ts_ft, y_pred))

In [0]:
# scores = model2.evaluate(ts_data, ts_pt, verbose=0)

# print("Accuracy: %.2f%%" % (scores[1]*100))
# y_pred = model2.predict(ts_data, batch_size=64, verbose=1)
# y_pred =(y_pred>0.5)

# print(classification_report(ts_pt, y_pred))
